In [ ]:
from pyspark.sql.types import StructType, StructField, StringType

# Extract schema names into a list
schema_names = sqlContext.sql("SHOW DATABASES").collect()
schemas = [row.databaseName for row in schema_names]

# Initialize an empty list for view definitions
all_view_definitions = []

for schema in schemas:
    # Get all views in the current schema
    views = sqlContext.sql(f"SHOW VIEWS IN `{schema}`").collect()

    for view in views:
        view_name = view.viewName
        # Get the SQL command to create the view
        view_definition = sqlContext.sql(f"SHOW CREATE TABLE `{schema}`.`{view_name}`").collect()

        # Assuming view_definition returns a single row with a single column
        if view_definition:
            create_statement = view_definition[0][0]
            all_view_definitions.append((schema, view_name, create_statement))

# Define the schema for the DataFrame
view_schema = StructType([
    StructField("Schema", StringType(), True),
    StructField("ViewName", StringType(), True),
    StructField("CreateStatement", StringType(), True)
])

# Convert the list to a DataFrame
views_df = spark.createDataFrame(all_view_definitions, view_schema)

# Write the DataFrame to a Hive table
views_df.write.mode('overwrite').saveAsTable("hive_metastore_view_definitions")
